# Introduction:
In this experiment, DSPY was utilized to automatically generate question and answer (QA) pairs from a given context, with a focus on processing Lithuanian language queries. The goal of the experiment was to explore the efficiency and accuracy of the DSPY framework in generating relevant QA pairs based on the provided context, with a challenge to do it with small nation language , which was not main training language for a model or is widely used by AI community. The experiment used the Gemma 2 27B Q4 model, and it aimed to assess how sensitive the generation process is in the choice of model.

### If DSPy doens't work for you on Windows, try to install with this code from a cell bellow

In [ ]:
!pip install git+https://github.com/stanfordnlp/dspy.git

In [10]:
import os
from typing_extensions import TypedDict, List
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from IPython.display import Image, display
from dspy import ChainOfThought
from pathlib import Path
from pydantic import BaseModel
import dspy
import chromadb
from dspy.retrieve.chromadb_rm import ChromadbRM
import pandas as pd

### Vectorstore initialization for infromation retrieval

In [12]:
model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {'device': 'cpu',
                  "trust_remote_code" : 'False'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

In [13]:
from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain_huggingface import HuggingFaceEmbeddings

langchain_embeddings = embeddings

ef = create_langchain_embedding(langchain_embeddings)

In [14]:
from dspy.retrieve.chromadb_rm import ChromadbRM

retriever_model = ChromadbRM(
    'Dspy_data_generation_final',
    './docs_final/docs/chroma',
    embedding_function=ef,
    k=5
)

In [36]:
results = retriever_model("Sandoris", k=5)
results

[{'id': 'bc961b8e-b7f8-4591-a833-289e55d1cfcb',
  'score': 0.24978500604629517,
  'long_text': 'Lituanica',
  'metadatas': {'Documento_pavadinimas': 'final_manoDataset.txt'}},
 {'id': 'bb8bc24e-234a-40d0-8cf6-b60a2183aed9',
  'score': 0.35074687004089355,
  'long_text': 'SANDORIŲ SAMPRATA IR RŪŠYS\nSandoriais laikomi asmenų veiksmai, kuriais siekiama sukurti, pakeisti arba panaikinti civilines teises\nir pareigas (CK 1.63 str. 1 d.).\nSandoriai gali būti vienašaliai, dvišaliai ir daugiašaliai. Vienašaliu laikomas sandoris, kuriam sudaryti\nbūtina ir pakanka vienos šalies valios (CK 1.63 str. 2–3 d.).\nIš vienašalio sandorio atsiranda pareigos jį sudariusiam asmeniui. Kitiems asmenims iš vienašalio sandorio pareigos atsiranda tik įstatymų nustatytais atvejais arba kai tie asmenys sutinka (CK 1.63 str. 4 d.).\nVienašaliams sandoriams prievoles ir sutartis reglamentuojančios teisės normos taikomos tiek, kiek tai\nneprieštarauja įstatymams ir vienašalio sandorio esmei (CK 1.63 str. 5 d.).\

### Initialization of a language model, which is local and simple test of it.

In [16]:
import dspy
lm = dspy.LM('ollama_chat/Gemma_2_27b_Q4_8k_ctx:latest', api_base='http://localhost:11434', api_key='',temperature = 0.0)
dspy.configure(lm=lm, rm =retriever_model)

In [17]:
lm(messages=[{"role": "user", "content": "pasakyk kad tai testas ir pridek dar kanors lietuviskai!"}])  # => ['This is a test!']

['Tai testas.\n\n**Kanorai:**\n\n* **Labas:** Sveiki\n* **Ačiū:** Ačiū\n* **Prašau:** Prašom\n* **Kiek kainuoja?** Kiek tai kainuoja?\n* **Kur yra tualetas?** Kur yra WC?\n\n\nAr norite, kad patikrintumem jūsų žinias apie lietuvių kalbą? \n']

### Queries , with whom will be information retrieved from a Chroma vectorstore

In [21]:
questions = [ 
    "Viešieji juridiniai asmenys", "Privatieji juridiniai asmenys", "Teisė į juridinio asmens pavadinimą",
    "Juridinių asmenų steigimas", "Juridinio asmens įstatai", "Juridinio asmens registravimas", 
    "bendrovės įstatinis kapitalas", "Juridinių asmenų registro duomenys neatlygintinai teikiami", 
    "Juridinių asmenų teisnumas", "Licencijavimo taisyklės", "Atsisakymas išduoti licenciją", 
    "Juridinio asmens organai", "Juridinių asmenų rūšys", "Akcinė bendrovė", "Ūkinė bendrija", 
    "Individuali įmonė", "Individualios įmonės steigimas", "Individualios įmonės samprata", 
    "Individualios įmonės savininkas turi teisę", "Individualios įmonės turtas", "Ūkinės bendrijos", 
    "Akcinė bendrovė", "uždaroji akcinė bendrovė", "Bendrovė", "Akcininkai", "Bendrovės įstatai", 
    "Turtinės ir neturtinės akcininkų teisės", "Bendrovės organai", "Visuotinis akcininkų susirinkimas", 
    "Stebėtojų tarybos kompetencija", "Stebėtojų tarybos sprendimų priėmimas", "Stebėtojų taryba", 
    "Valdybos kompetencija", "Bendrovės vadovas", "Akcijos", "privilegijuotosios akcijos", 
    "Paprastosios akcijos", "Obligacijos", "Dividendai", "Bendrovės reorganizavimas", 
    "Bendrovės pertvarkymas", "Bendrovės likvidavimas", "savivaldybės įmonės tikslas", 
    "Valstybės įmonės savininkė", "Žemės ūkio bendrovė", "Žemės ūkio bendrovės", "Bendrovės steigėjai", 
    "Bendrovės steigimo dokumentai", "Mažoji bendrija", "Ribota civilinė atsakomybė", 
    "Mažosios bendrijos steigimas", "Mažosios bendrijos registravimas", "Mažosios bendrijos narių teisės ir pareigos", 
    "Įnašai į mažąją bendriją", "Mažosios bendrijos nario savanoriškas pasitraukimas", 
    "Mažosios bendrijos organai", "Mažosios bendrijos vadovas", "Mažosios bendrijos apskaita", 
    "Mažosios bendrijos finansinė atskaitomybė", "Mažosios bendrijos narių gaunamos išmokos", "Daiktai", 
    "Daiktinė teisė", "Daikto valdymas", "Valdymo teisės objektas", "Nuosavybės teisė", 
    "Nuosavybės teisės subjektai", "Nuosavybės teisės įgijimo pagrindai", "Nuosavybės teisės pagrindinės teisės", 
    "Disponuoti", "Radinys", "Bendroji dalinė nuosavybė", "Savininko teisių apsauga", "savininko nuosavybę", 
    "Niekas neturi teisės", "Negatorinis ieškinys", "Turto patikėjimo teisės atsiradimo pagrindas", 
    "Servitutas", "Servitutą gali nustatyti", "teismo sprendimas, administracinis aktas.", "Uzufruktas", 
    "Uzufrukto objektas", "Hipoteka", "Daikto sulaikymas", "Paveldėjimas", "Įpėdiniai", "Testamentų rūšys", 
    "Įpėdinių pagal įstatymą eilės", "Konsensualinis sandoris", "prievolių teisė", "Kauzalinis sandoris", 
    "Abstraktus sandoris", "Sutarties samprata", "Esminis sutarties požymis", "Valios teorija", 
    "Ekonominė teorija", "Sutarties laisvės principas", "Socialinė teorija", "Pasitikėjimo teorija", 
    "sąžininga dalykinė praktika", "Sąžiningumas", "Moralinis sutarties aspektas", "sutartis negalioja", 
    "Sutarties elementai", "Sutarčių rūšys", "dvišalės sutartys", "vienašalės sutartys", "atlygintinės sutartys", 
    "neatlygintinės sutartys", "konsensualinės sutartys", "realinės sutartys", "vienkartinio įvykdymo sutartys", 
    "tęstinio vykdymo sutartys", "Viešoji sutartis", "Sutarčių sudarymas", "konsesualizmo principas", 
    "esminės sutarties sąlygos", "įprastinės sutarties sąlygos", "atsitiktinės sutarties sąlygos", 
    "Konfidencialumo pareiga", "Preliminarioji sutartis", "Žinojimo prezumpcija", "Oferta", "Ofertos galiojimas", 
    "Ofertos atšaukimas", "Oferta netenka galios", "Viešoji oferta", "Akceptas ir jo formos", "Akceptas", 
    "Akceptavimo terminas", "Pavėluotas akceptas", "Akcepto atšaukimas", "Sutarties sudarymo momentas", 
    "Sutarties sudarymo vieta", "Viešųjų sutarčių sudarymas", "nestandartinių sutarčių sąlygų prieštaravimas", 
    "Vartojimo sutarčių sąlygų ypatumai", "Vartotojas", "civilinė atsakomybė", "Nesąžiningos sutarčių sąlygos", 
    "Sutarties galia", "Sutarties privalomumo principas", "Sutarties galia tretiesiems asmenims", 
    "Sutarties forma", "Sutarčių aiškinimas", "Sutarčių aiškinimo taisyklės", "Sutarties sąlygų rūšys", 
    "Sutarties kaina", "Sutarties vykdymo principai", "Sutartis neapibrėžtam terminui", 
    "Sutartinių įsipareigojimų vykdymas pasikeitus aplinkybėms", "Sutarčių neįvykdymo teisinės pasekmės", 
    "Sutarties vykdymo sustabdymas", "Įvykdymo trūkumų pašalinimas", "Papildomas terminas", "Palūkanos", 
    "Atsakomybę naikinančios sąlygos", "Nenugalima jėga", "Reikalavimas įvykdyti sutartį", 
    "Bauda už įpareigojimo įvykdyti sutartinę prievolę natūra nevykdymą", "Sutarties nutraukimas", 
    "Pranešimas apie sutarties nutraukimą", "Sutarties nutraukimo teisinės pasekmės", "Restitucija", 
    "Sutarties pakeitimas", "Sutarties negaliojimas", "santykinis sutarties negaliojimas", 
    "absoliutus sutarties negaliojimas", "Dalinis sutarties negaliojimas", "Teisė pareikšti ieškinį dėl sutarties negaliojimo", 
    "Šalis gali atsisakyti sutarties", "Teismas gali pakeisti sutartį", 
    "Lituanika", "S. Darius ir S. Girėnas", "Arvydas Sabonis", "Laikomas geriausiu Lietuvos krepšininku",
    "Lozoraitis Stasys", "Mykolas Romeris", "Jonas Basanavičius", "Vincas Kudirka", "Antanas Smetona",
    "Molotovo–Ribentropo paktas", "Lietuvos laisvės lyga", "Adomas Mickevičius", "Radvilos", "Liublino unija",
    "Rusnė", "Švėkšna", "Fotosintezė", "Archimedas", "Archimedo dėsnis", "Olimpo Dievai", "Dzeusas", "Poseidonas",
    "Afroditė", "Atėnė", "Apolonas", "Kastis Torrau", "Aleksandras Stulginskis", "Adolfas Ramanauskas-Vanagas",
    "CIVILINĖS TEISĖS REGULIAVIMO DALYKAS", "Civilinis kodeksas", "Civilinio kodekso reguliavimo principai",
    "Sutarties laisvės principas", "Nuosavybės neliečiamumo principas", "Civilinių santykių subjektų lygiateisiškumo principas",
    "Teisinio apibrėžtumo principas", "Proporcingumo principas", "Neleistinumo piktnaudžiauti teise",
    "Civilinis teisinis santykis", "Civilinis teisnumas", "Civilinis veiksnumas", "civilinis teisinis santykis",
    "Civilinio teisinio santykio turinys", "Juridinis faktas", "Teisinis poelgis", "Teisinis aktas", "Sandoris",
    "vienašalis sandoris", "Sąlyginis sandoris", "Vienašalio sandorio sudarymo vieta", "Rašytinės formos sandorių pasirašymas",
    "Niekiniai sandoriai", "Nuginčijamas sandoris", "negaliojantis sandoris", "Pinigai", "Akcija", "Vertybinis popierius",
    "Obligacija", "Čekis", "Vekselis", "Konosamentas", "Indėlio liudijimas", "Daiktai, kaip civilinių teisių objektai",
    "Žemės sklypas ir kiti ištekliai", "Įmonės ir turtiniai kompleksai", "Intelektinės veiklos rezultatai",
    "Turtinės teisės", "Asmeninės neturtinės teisės", "Komercinė (gamybinė) ir profesinė paslaptis", "Garantiniai",
    "Ieškinio senatis", "Naikinamasis", "Įgyjamasis", "Atnaujinamasis", "Terminas", "Ieškinio senaties terminų rūšys",
    "Fizinis asmuo", "Juridinis asmuo", "Fizinio asmens pripažinimas neveiksniu", "Fizinio asmens nuolatinė gyvenamoji vieta",
    "Teisė į vardą", "Teisės į vardą gynimas", "Teisė į atvaizdą", "Nuotrauka", "Fizinio asmens privatus gyvenimas neliečiamas",
    "Teisė į privatų gyvenimą ir jo slaptumą", "Asmens garbės ir orumo gynimas", "Neleistinumas apriboti fizinio asmens laisvę",
]


In [22]:
len(questions)

258

### Information generation module
    """
    A module that retrieves relevant context for a given query and generates trivia-style questions and answers in Lithuanian.

    Attributes:
        retrieve (dspy.Retrieve): A retrieval mechanism to fetch the top context passages related to the input query.

    Methods:
        forward(query): 
            Retrieves the most relevant context passages for a given query and then generates questions and answers in Lithuanian 
            based on the retrieved context. It returns a list of dictionaries, each containing the context, generated question, 
            and corresponding answer.
    """



In [23]:
class GenerateQuestion(dspy.Signature):
    """Sudaryk klausimą iš gautos informacijos ir pateikto atsakymo. Klausimas Lietuvių kalba! """
    information = dspy.InputField()
    question = dspy.OutputField()
    answer = dspy.OutputField(desc= "trumpas atsakymas Lietuvių kalba")


class GenerateTrivia(dspy.Module):
    def __init__(self):
        self.retrieve = dspy.Retrieve(k= 1)

    def forward(self, query):
        contexts = self.retrieve(query).passages
        answers = []
        for k in contexts:
            output = dspy.ChainOfThought(GenerateQuestion) (information = k)
            answers.append({
                "context": k ,
                "question" : output.question,
                "answer" : output.answer
            })
        return answers    

##### Iterate over a list of questions and generate  questions and answers for each, add the generated results to 'generated_list', or print a message if no results are found.

In [24]:
generated_list = []
klausimu_generatorius = GenerateTrivia()
for q in questions:
    result = klausimu_generatorius(query=q)
    if result:  # Check if the result is not empty
        generated_list.extend(result)
    else:
        print(f"No results for query: {q}")

#### Generated dataframe
If DSPy program doesnt make a question and answer pair from a given context or some error appears, than that records isn't added into a dataframe.
This functionality is very sensitive to different LLM models.

In [25]:
Retrieva_QA_creation_df = pd.DataFrame(generated_list)
Retrieva_QA_creation_df = Retrieva_QA_creation_df.drop_duplicates(subset=['question', 'answer'], keep="first")
Retrieva_QA_creation_df

,context,question,answer
0,JURIDINIAI ASMENYS\nPagrindinės sąvokos\nJurid...,Kokiu tikslu steigiamams viešieji ir privatūs ...,Viešieji juridiniai asmenys steigiamai tenkint...
2,Teisė į juridinio asmens pavadinimą\nNeleidžia...,"Ką gali padaryti juridinis asmuo, kurio teisė ...",Jis gali kreiptis į teismą ir reikalauti nutra...
5,Juridinio asmens registravimo pažymėjimas ir k...,Kokia informacija turi būti pateikta registruo...,"Registruojant juridinį asmenį, reikia pateikti..."
6,Akcinė bendrovė ir uždaroji akcinė bendrovė\nA...,"Koks yra minimalus įstatinis kapitalas, reikal...",Uždarosios akcinės bendrovės įstatinis kapital...
9,Licencijavimo taisyklės\nKiekvienai įstatymų n...,Kiek laiko turi būti išduota licencija verstis...,"Per trisdešimt dienų nuo dokumentų, reikalingų..."
...,...,...,...
249,Fizinio asmens nuolatinė gyvenamoji vieta\nFiz...,Kas yra fizinio asmens nuolatinė gyvenamoji vi...,Fizinis asmuo pripažįstamas turinčiu nuolatinę...
251,"Teisės į vardą gynimas\nFizinis asmuo, kurio t...","Ką turi teisę daryti fizinis asmuo, kurio vard...",Jis gali kreiptis į teismą ir reikalauti nutra...
252,Teisė į atvaizdą\nFizinio asmens nuotrauka (jo...,Kada galima fotografuoti ar kitaip fiksuoti as...,"Asmens sutikimo nereikia, jeigu šie veiksmai y..."
254,Teisė į privatų gyvenimą ir jo slaptumą\nFizin...,Kokie yra pagrindiniai asmens privatumo pažeid...,Neteisėtas ėjimas į asmens gyvenamąsias ir kit...


In [26]:
len(Retrieva_QA_creation_df)

113

In [28]:
Retrieva_QA_creation_df.to_csv("generated_CQA")

# Conclusion:
The experiment demonstrated that DSPY, when paired with the Gemma 2 27B Q4 model, can effectively generate question and answer pairs from a given context. However, the results underscored that the information module's performance is highly sensitive to the specific model used. This indicates that model selection plays a crucial role in determining the quality and relevance of the generated QA pairs. Additionally, the experiment faced the challenge of processing Lithuanian, a language spoken by a small population and not widely supported in natural language processing tools. Despite this, the model performed reasonably well in generating accurate QA pairs. Further investigation into optimizing the choice of model and refining the information retrieval process can improve the overall performance of the system.